<a href="https://colab.research.google.com/github/dodor3030/303030/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q pyngrok flask_ngrok
!pip install -q flask line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 9.5 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK'))
    ngrok.connect(5000)
    run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,StickerMessage,ImageMessage,VideoMessage,LocationMessage
)

app = Flask(__name__)
def index():
  return 'hello'

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body,signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == 'confirm':
          template = ConfirmTemplate(
              text="你喜歡葬送的芙莉蓮嗎?",
              actions=[
                  MessageAction(label="是", text="我超喜歡芙莉蓮!"),
                  MessageAction(label="否", text="其實我非常喜歡，只是我要傲嬌的說不")
              ]
          )
        elif action == 'carousel':
          carousel_template = CarouselTemplate(
            columns=[
              CarouselColumn(
                  thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/7/7d/Summer_Wars_poster.jpg',
                  title = '夏日大作戰',
                  text = '細田守導演的日本科幻暨浪漫電影',
                  actions = [
                      URIAction(label='維基百科',uri='https://zh.wikipedia.org/zh-tw/%E5%A4%8F%E6%97%A5%E5%A4%A7%E4%BD%9C%E6%88%B0'),
                      MessageAction(label='我要看',text='我要看夏日大作戰')
                  ]
              ),
              CarouselColumn(
                  thumbnail_image_url='https://upload.wikimedia.org/wikipedia/zh/thumb/4/4f/Castle_of_Cagliostro_poster.png/250px-Castle_of_Cagliostro_poster.png',
                  title = '魯邦三世 卡里奧斯特羅城',
                  text = '宮崎駿導演的日本動畫冒險戲劇電影',
                  actions = [
                      URIAction(label='維基百科',uri='https://zh.wikipedia.org/zh-tw/%E9%AD%AF%E9%82%A6%E4%B8%89%E4%B8%96_%E5%8D%A1%E9%87%8C%E5%A5%A7%E6%96%AF%E7%89%B9%E7%BE%85%E4%B9%8B%E5%9F%8E'),
                      MessageAction(label='投票',text='我投魯邦三世 卡里奧斯特羅城一票!')
                  ]
                  )
              ]
            )
          reply = TemplateMessage(
              alt_text = "這是輪播視窗",
              template = carousel_template
          )

        else:
          reply = TextMessage(text='Thanks!')


        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                    reply
                ]
            )
        )

ngrok_start() #for colab
if __name__ == "__main__":
    app.run()



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://f321-107-178-221-237.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [13/May/2025 13:46:50] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 13:46:50] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 13:46:57] "GET /callack HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 13:47:03] "GET /callback HTTP/1.1" 405 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 13:47:14] "POST /callback HTTP/1.1" 200 -
